In [1]:
import pandas as pd
import gurobipy as gp
from gurobipy import GRB

In [21]:
garages = [
    'Winchester',
    'Andover',
    'Basingstoke',
    'Poole',
    'Woking',
    'Newbury',
    'Portsmouth',
    'Alresford',
    'Salisbury',
    'Guildford',
    'Alton',
    'Weybridge',
    'Dorchester',
    'Bridport',
    'Weymouth',
    'Portland',
    'Chichester',
    'Petersfield',
    'Petworth',
    'Midhurst',
    'Reading',
    'Southampton',
    'Bournemouth',
    'Henley',
    'Maidenhead',
    'Fareham',
    'Romsey',
    'Ringwood']

in_staff  = dict(zip(garages, [7, 6, 2, 14, 10, 24, 6, 8, 5, 8, 
                              7, 5, 6, 11, 4, 3, 5, 21, 6, 3, 30, 
                               25, 19, 7, 12, 4, 2, 2]))
in_space  = dict(zip(garages, [8, 6, 3, 9, 9, 15, 7, 7.5, 5, 10, 
                              8, 6.5, 7.5, 8, 5, 3.5, 5.5, 12, 5.5, 
                               3.6, 29, 16, 10, 6, 8,6, 2.5, 3.5]))
in_pop1   = dict(zip(garages, [10, 20, 40, 20, 10, 15, 50, 5, 10, 30,
                              7, 9, 10, 8, 10, 3, 8, 6, 2, 3, 120, 110, 
                               90, 5, 7, 1, 1, 2]))
in_pop2   = dict(zip(garages, [12, 30, 40, 25, 10, 13, 40, 8, 10, 35,
                              8, 12, 10, 10, 10, 20, 10, 6, 2, 3, 80, 
                               80, 22, 7, 10, 1, 1, 2]))
in_alpha  = dict(zip(garages, [8.5, 9, 2, 10, 11, 25, 8.5, 9, 5, 9.5, 
                              3, 8, 7.5, 10, 7.5, 2, 7, 15, 8,2.5,35,
                               27,25,8.5,12,7.5,2.5,1.9]))
in_beta   = dict(zip(garages, [4, 4.5, 1.5, 6, 5, 1.9 , 3, 4, 2.5, 4.5,
                              2, 4.5, 4, 6, 3.5, 1.5, 3.5, 8, 5, 1.5, 20,
                               12, 13, 4.5, 7, 3.5, 1, 1.2]))
out_alpha = dict(zip(garages, [2, 2.3, 0.8, 2.6, 2.4, 8, 2.5, 2.1, 2, 2.05,
                              1.9, 1.8, 1.5, 2.2, 1.8, 0.9, 1.2, 6, 1.5, 0.8,
                               7, 6.5, 5.5, 1.2, 4.5, 1.1, 0.4, 0.3]))
out_beta  = dict(zip(garages, [0.6, 0.7, 0.25, 0.86, 1, 2.6, 0.9, 0.85, 0.65, 0.75,
                              0.7, 0.63, 0.45, 0.65, 0.62, 0.35, 0.45, 0.25, 0.55,
                              0.2, 2.5, 3.5, 3.1, 0.48, 2, 0.48, 0.1, 0.09]))
out_prof  = dict(zip(garages, [1.5, 1.6, 0.5, 1.9, 2, 4.5, 1.6, 2, 0.9, 1.7,
                              0.5, 1.4, 1.45, 2.2, 1.6, 0.5, 1.3, 2.9, 1.55,
                              0.45, 8, 5.4, 4.5, 2, 2.3, 1.7, 0.55, 0.4]))


In [54]:
efficiency = {}

for k in garages:

    model = gp.Model('Efficiency Analysis ' + k)

    # add vars
    x = model.addVars(garages,
                     name='x')
    w = model.addVar(name='w')

    # objective function
    model.setObjective(w,
                      GRB.MAXIMIZE)

    # constraints
    # input: staff
    model.addConstr(gp.quicksum(in_staff[j]*x[j] for j in garages) <= in_staff[k],
                   name='in_staff')

    # input: show room space
    model.addConstr(gp.quicksum(in_space[j]*x[j] for j in garages) <= in_space[k],
                   name='in_space')

    # input: population category 1
    model.addConstr(gp.quicksum(in_pop1[j]*x[j] for j in garages) <= in_pop1[k],
                   name='in_pop1')

    # input: population category 2
    model.addConstr(gp.quicksum(in_pop2[j]*x[j] for j in garages) <= in_pop2[k],
                   name='in_pop2')

    # input: alpha enquiries
    model.addConstr(gp.quicksum(in_alpha[j]*x[j] for j in garages) <= in_alpha[k],
                   name='in_alpha')

    # input: beta enquiries
    model.addConstr(gp.quicksum(in_beta[j]*x[j] for j in garages) <= in_beta[k],
                   name='in_beta')


    # output: alpha revenues
    model.addConstr(gp.quicksum(out_alpha[j]*x[j] for j in garages) >= out_alpha[k]*w,
                   name='out_alpha')

    # output: beta revenues
    model.addConstr(gp.quicksum(out_beta[j]*x[j] for j in garages) >= out_beta[k]*w,
                   name='out_beta')

    # output: profit
    model.addConstr(gp.quicksum(out_prof[j]*x[j] for j in garages) >= out_prof[k]*w,
                   name='out_prof')

    model.update()
    #model.write('Efficiency Analysis ' + k + '.lp')
    model.Params.LogToConsole = 0
    model.optimize()

    efficiency[k] = model.ObjVal

    print('{} Efficiency = {:.3f}'.format(k, 1/model.ObjVal))
    
print('\nInneficient Garages')
for k in garages:
    if efficiency[k] > 1:
        print('\t{}'.format(k))

Winchester Efficiency = 0.835
Andover Efficiency = 0.917
Basingstoke Efficiency = 1.000
Poole Efficiency = 0.864
Woking Efficiency = 0.845
Newbury Efficiency = 1.000
Portsmouth Efficiency = 1.000
Alresford Efficiency = 1.000
Salisbury Efficiency = 1.000
Guildford Efficiency = 0.802
Alton Efficiency = 1.000
Weybridge Efficiency = 0.854
Dorchester Efficiency = 0.867
Bridport Efficiency = 0.982
Weymouth Efficiency = 1.000
Portland Efficiency = 1.000
Chichester Efficiency = 0.825
Petersfield Efficiency = 1.000
Petworth Efficiency = 0.988
Midhurst Efficiency = 0.889
Reading Efficiency = 0.984
Southampton Efficiency = 1.000
Bournemouth Efficiency = 1.000
Henley Efficiency = 1.000
Maidenhead Efficiency = 1.000
Fareham Efficiency = 1.000
Romsey Efficiency = 1.000
Ringwood Efficiency = 0.908

Inneficient Garages
	Winchester
	Andover
	Poole
	Woking
	Guildford
	Weybridge
	Dorchester
	Bridport
	Chichester
	Petworth
	Midhurst
	Reading
	Ringwood
